# Template

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')


# Create subplots and mention plot grid size
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.01, subplot_titles=('OHLC', ''), 
               row_width=[0.2, 0.7])

# Row 1
## 1. OHLC candlestick 
fig.add_trace(go.Candlestick(x=df["Date"], open=df["AAPL.Open"], high=df["AAPL.High"],
                low=df["AAPL.Low"], close=df["AAPL.Close"], name="OHLC"), 
                row=1, col=1
)

## 2. MA5, MA20, MA60
fig.add_trace(go.Scatter(x=df["Date"], y=df["AAPL.Open"],
                    mode='lines',
                    name='lines'),
                    row=1, col=1
)

color_map = ["#00995c" if rise else "#eb2409"  for rise in df["AAPL.Open"] <= df["AAPL.Close"]]
# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=df['Date'], y=df['AAPL.Volume'],
                    marker_color=color_map,
                    showlegend=False), row=2, col=1)

# Do not show OHLC's rangeslider plot 
fig.update(layout_xaxis_rangeslider_visible=False)


fig.update_layout(
    template = 'plotly_white',
    xaxis=dict(
        rangebreaks=[
            dict(bounds=["sat", "mon"]), #hide weekends
            dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
    ]),
    xaxis2=dict(
                rangebreaks=[
                    dict(bounds=["sat", "mon"]), #hide weekends
                    dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
                    ],
                showgrid=False,
                linewidth=1,
                linecolor='black',
                mirror=True
                ))

fig.show()

# Final

In [294]:
import psycopg2

import os
import json
from talib import abstract

In [295]:
def get_stock_price(date, stock_ids):
    conn_params = {
    "host" : "localhost",
    "database" : "Fin_proj",
    "user" : "postgres",
    "password" : "nckumark"
    }
    sql = """   SELECT *
                FROM daily_stock_price
                WHERE date >= (%s) AND \
                stock_id = ANY(%s);
                """
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect(**conn_params)
        # create a new cursor
        cur = conn.cursor()
        # execute the SQL statement
        cur.execute(sql, (date, stock_ids))
        rows = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return pd.DataFrame(rows, columns=column_names)

In [296]:
date = '2022/01/01'
stock_ids = '3008'
Stock_df = get_stock_price(date, [stock_ids])

In [297]:
def calculate_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    df = df.sort_index(inplace=False)
    df['MA5_close']=df['close'].rolling(5).mean()
    df['MA20_close']=df['close'].rolling(20).mean()
    df['MA60_close']=df['close'].rolling(60).mean()

    df['volume_unit'] = df['volume'] /1000
    df['MA5_vol']=df['volume_unit'].rolling(5).mean()
    df['MA20_vol']=df['volume_unit'].rolling(20).mean()
    df['MA60_vol']=df['volume_unit'].rolling(60).mean()
    df = df.reset_index()
    return df

In [298]:
plot_df = calculate_data(Stock_df)

In [309]:
def daily_plot(df):
    stock_id = df['stock_id'].values[0]
    # Filter out empty date
    gap_date = set(pd.date_range(start=df.date[0], end=df.date.tolist()[-1], freq='d')) - set(df.date)
    gap_dates = pd.Series(list(gap_date)).dt.strftime('%Y-%m-%d').tolist()

    # Create subplots and mention plot grid size
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01, subplot_titles=('', ''), 
                row_width=[0.2, 0.7])

    # Row 1
    ## 1. OHLC candlestick 
    fig.add_trace(go.Candlestick(x=df["date"], open=df["open"], high=df["high"],
                    low=df["low"], close=df["close"], name="OHLC", whiskerwidth=0) ,
                    row=1, col=1)
    color_map = ["#00995c" if rise else "#eb2409"  for rise in df["open"] < df["close"]]
    fig.update_traces(selector=dict(type="candlestick"), 
                    showlegend = False,
                    hoverlabel = dict(
                        bgcolor = color_map),
                    )
    ## 2. MA5, MA20, MA60
    col_names = df.columns[df.columns.str.contains('date|_close')].tolist()
    melt_MA_df = pd.melt(df[col_names], id_vars='date', value_vars=col_names[1:], var_name= 'MA')
    melt_MA_df['value'] = melt_MA_df['value'].round(2)
    color_dict = dict(zip(col_names[1:], ['rgb(240, 191, 76)', 'rgb(91, 209, 215)', 'RoyalBlue']))
    for MA in col_names[1:]:
        close_sr = melt_MA_df.loc[melt_MA_df.MA == MA, 'value']
        date_sr = melt_MA_df.loc[melt_MA_df.MA == MA, 'date']
        ma_trace = go.Scatter(x=date_sr, y=close_sr,
                            name=MA,
                            line=dict(color=color_dict[MA], width=1),
                            hovertemplate= MA + ':%{y}<extra></extra>',
                            hoverlabel=dict(
                                font_size=8, 
                                font_family="Rockwell",
                                namelength = 0),
                            )
        fig.add_trace(ma_trace, row=1, col=1)   
    # Row 2
    ## 1. Volume(total value) bar chart
    # Bar trace for volumes on 2nd row without legend
    fig.add_trace(go.Bar(x=df['date'], y=df['volume_unit'],
                        marker_color=color_map,
                        showlegend=False,
                        hovertemplate='Vol:%{y:,.2s}<extra></extra>',)
                        , row=2, col=1)
    ## 2. MA5, MA20, MA60
    col_names = df.columns[df.columns.str.contains('date|_vol')].tolist()
    melt_MA_df = pd.melt(df[col_names], id_vars='date', value_vars=col_names[1:], var_name= 'MA')
    melt_MA_df['value'] = melt_MA_df['value'].round(2)
    color_dict = dict(zip(col_names[1:], ['rgb(240, 191, 76)', 'rgb(91, 209, 215)', 'RoyalBlue']))
    for MA in col_names[1:]:
        close_sr = melt_MA_df.loc[melt_MA_df.MA == MA, 'value']
        date_sr = melt_MA_df.loc[melt_MA_df.MA == MA, 'date']
        ma_trace = go.Scatter(x=date_sr, y=close_sr,
                            name=MA,
                            showlegend=False,
                            line=dict(color=color_dict[MA], width=1),
                            hovertemplate= MA + ':%{y:,.2s}<extra></extra>',
                            hoverlabel=dict(
                                font_size=8, 
                                font_family="Rockwell",
                                namelength = 0),
                            )
        fig.add_trace(ma_trace, row=2, col=1)  

    # # Do not show OHLC's rangeslider plot 
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.update_layout(
        title = dict(
            text = f'{stock_id}',
            font_size = 18,
            yanchor='top',
            y=0.92,
            xanchor="left",
            x=0.045,
        ),
        template = 'plotly_dark',
        autosize=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=0.94,
            xanchor="left",
            x=0,
            bgcolor='rgba(0,0,0,0)',
            font=dict(
                family="Courier",
                size=12,
                #color="black"
                ),
            ),
        hovermode="x",
        xaxis=dict(rangebreaks=[dict(values=gap_dates)]),
        xaxis2=dict(
            rangebreaks=[dict(values=gap_dates) ],
            showgrid=False,
            linewidth=1,
            linecolor='black',
            mirror=True),
        margin=dict(
            l=50,
            r=50,
            b=50,
            t=50,
            pad=1
        ),
    ) 
    fig.update_xaxes(tickformat='%m/%d<br>%Y', nticks=25, tickfont=dict(size=9))
    fig.update_yaxes(side='right')
    fig.show()

In [308]:
daily_plot(plot_df)

In [310]:
daily_plot(plot_df)